In [ ]:
import requests
import time
import json
import os
import tempfile

from pprint import pprint
import SimpleITK as sitk

base_url = 'http://127.0.0.1:8001'
api_key = ''

algorithm_name = "
"
api_dicom_location = '{0}/api/dicomlocation'.format(base_url)
api_dataset = '{0}/api/dataset'.format(base_url)
api_dataset_ready = '{0}/api/dataset/ready'.format(base_url)
api_data_object = '{0}/api/dataobject'.format(base_url)
api_trigger = '{0}/api/trigger'.format(base_url)
api_algorithm = '{0}/api/algorithm'.format(base_url)
api_download = '{0}/api/dataobject/download'.format(base_url)

In [ ]:
# Get the algorithm and the default settings
algorithm = None
r = requests.get(api_algorithm, headers={'API_KEY': api_key})
if r.status_code == 200:
    for a in r.json():
        pprint(a)
        if a['name'] == algorithm_name:
            algorithm = a
    print("")
    print("Look's Good!")
else:
    print("Oops, something went wrong. Ensure the service is running at the base_url configured and that the API Key has been generated and set in api_key.")

In [ ]:
# Create a new Dataset
dataset = None
data = {}
r = requests.post(api_dataset, headers={'API_KEY': api_key}, data=data)
if r.status_code >= 200:
        dataset = r.json()
        
pprint(dataset)

In [ ]:
# Use the phantom dicom data, both as primary and secondary
path_to_dicom = f"../../dicom/data/phantom/CT"
load_path = sitk.ImageSeriesReader().GetGDCMSeriesFileNames(path_to_dicom)
image = sitk.ReadImage(load_path)

tmp_dir = tempfile.mkdtemp()
tmp_file = os.path.join(tmp_dir, "img.nii.gz")
sitk.WriteImage(image, tmp_file)

data_object = None

# Add the primary image
with open(tmp_file,'rb') as file:

    data = {'dataset': dataset['id'],
            'type': 'FILE',
            'meta_data': json.dumps({"type": "primary",}),
            'file_name': 'primary.nii.gz'}
    
    r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data, files={'file_data':file})
    if r.status_code >= 200:
            primary_data_object = r.json()
            
# Add a child struct in which to find points for primary
path_to_struct = f"../../dicom/data/phantom/masks/Test_MANDIBLE_PRI.nii.gz"
with open(path_to_struct,'rb') as file:

    data = {'dataset': dataset['id'],
            'type': 'FILE',
            "parent": primary_data_object['id'],
            'meta_data': json.dumps({"type": "contour","name": "Struct_1"}),
            'file_name': 'struct.nii.gz'}
    
    r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data, files={'file_data':file})
    if r.status_code >= 200:
            secondary_contour_data_object = r.json()
            
# Add the secondary Image
with open(tmp_file,'rb') as file:

    data = {'dataset': dataset['id'],
            'type': 'FILE',
            'meta_data': json.dumps({"type": "secondary",}),
            'file_name': 'secondary.nii.gz'}
    
    r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data, files={'file_data':file})
    if r.status_code >= 200:
            secondary_data_object = r.json()


            
# Add a child struct in which to find points for primary
path_to_struct = f"../../dicom/data/phantom/masks/Test_MANDIBLE_PRI.nii.gz"
with open(path_to_struct,'rb') as file:

    data = {'dataset': dataset['id'],
            'type': 'FILE',
            "parent": secondary_data_object['id'],
            'meta_data': json.dumps({"type": "contour","name": "Struct_1"}),
            'file_name': 'struct.nii.gz'}
    
    r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data, files={'file_data':file})
    if r.status_code >= 200:
            primary_contour_data_object = r.json()
            
# Take a look at the dataset objects output
r = requests.get('{0}/{1}'.format(api_dataset, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    dataset = r.json()
    pprint(dataset)

In [ ]:
# Get the algorithm and the default settings
algorithm = None
r = requests.get(api_algorithm, headers={'API_KEY': api_key})
if r.status_code == 200:
    for a in r.json():
        if algorithm_name in a['name']:
            algorithm = a
        print(a)

In [ ]:
settings = algorithm['default_settings']
settings['includePointsMode'] = "BOUNDINGBOX"
settings['intensityRange'] = [-200, 200]

In [ ]:
settings

In [ ]:
# Trigger the algorithm with our dataset containing the data object
data={'dataset': dataset['id'],
     'algorithm': algorithm['name'],
     'config': json.dumps(settings)}
r = requests.post(api_trigger, headers={'API_KEY': api_key}, data=data)

if r.status_code == 200:
    # Poll the URL given to determine the progress of the task
    poll_url = '{0}{1}'.format(base_url, r.json()['poll'])
    
    while(1):
        r = requests.get(poll_url, headers={'API_KEY': api_key})
        status = r.json()
        print(status)

        if status['state'] == 'SUCCESS' or status['state'] == 'FAILURE':
            break

        time.sleep(1)
else:
    print(r.json())
    
print('Algorithm Processing Complete')

In [ ]:
# Take a look at the dataset objects output
r = requests.get('{0}/{1}'.format(api_dataset, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    dataset = r.json()
    pprint(dataset)

    for d in dataset['output_data_objects']:
        r = requests.get('{0}/{1}'.format(api_download, d['id']), headers={'API_KEY': api_key})
        filename = r.headers['Content-Disposition'].split('filename=')[1]
        print('Downloading to: {0}'.format(filename))
        open(filename, 'wb').write(r.content)

In [ ]:
import pandas as pd
ds = pd.read_csv("primary_Struct_1_match.csv", header=None)
ds